#데이터 불러오기  

In [ ]:
pip install dfply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 612 kB 5.0 MB/s 


In [ ]:
import os 
import pandas as pd
import numpy as np 
from dfply import *

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Data/big_contest')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Data/big_contest


In [ ]:
f_name = sorted(os.listdir())
print(f_name)

['loan_result.csv', 'log_data.csv', 'result.csv', 'test.csv', 'train.csv', 'user_spec.csv']


In [ ]:
loan = pd.read_csv('loan_result.csv')
spec = pd.read_csv('user_spec.csv')
result = pd.read_csv('result.csv')

#데이터 전처리

In [ ]:
spec.isnull().sum(0) #열단위 결측값 확인

application_id                               0
user_id                                      0
birth_year                               12961
gender                                   12961
insert_time                                  0
credit_score                            105115
yearly_income                               90
income_type                                 85
company_enter_month                     171760
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn              587461
personal_rehabilitation_complete_yn    1203354
existing_loan_cnt                       198556
existing_loan_amt                       313774
dtype: int64

In [ ]:
spec1 = spec.copy()

In [ ]:
spec1 = spec1.sort_values(['user_id', 'insert_time'])

#결측값을 입력시간에 따라 처리하기위해 데이터 정렬

## 공통 결측값 포함 열 제거

In [ ]:
cond = spec1.isnull().sum(0) == 85
na85_col = spec1.columns[cond]
print(na85_col)

Index(['income_type', 'employment_type', 'houseown_type', 'desired_amount',
       'purpose'],
      dtype='object')


In [ ]:
na85_ind = []
for i in range(len(na85_col)):
  na85_ind.append(spec1[spec1[na85_col[i]].isnull()]['application_id'])

#결측치 개수가 85개인 application_id 추출

In [ ]:
res = [list(set(na85_ind[0]).intersection(i)) for i in na85_ind]
[len(i) for i in res]

#모든 application_id가 동일, 85개 행에서 공통적으로 na존재

[85, 85, 85, 85, 85]

In [ ]:
na85_apid = na85_ind[0]

#application_id 추출

In [ ]:
loan[loan['application_id'].isin(na85_apid)]

#loan데이터에서 na85의 application_id가 존재하지 않기때문에 제거

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied


In [ ]:
spec1 = spec1[~spec1['application_id'].isin(na85_apid)]

#85개 행 삭제 

## birth_year, gender

결측치 처리

In [ ]:
spec2 = spec1.copy()

In [ ]:
spec2.isnull().sum(0)

application_id                               0
user_id                                      0
birth_year                               12959
gender                                   12959
insert_time                                  0
credit_score                            105112
yearly_income                                5
income_type                                  0
company_enter_month                     171675
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198540
existing_loan_amt                       313752
dtype: int64

In [ ]:
spec2[spec2['birth_year'].isnull() & spec2['gender'].isnull()]

# birth_year, gender의 결측값경우 모두 동일행에 존재 

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
978188,273996,52,NaN,NaN,2022-04-12 18:02:28,660.0,39000000.0,EARNEDINCOME,202103.0,정규직,기타가족소유,70000000.0,대환대출,NaN,NaN,7.0,71000000.0
965121,1300509,52,NaN,NaN,2022-04-13 18:05:01,660.0,39000000.0,EARNEDINCOME,202103.0,정규직,기타가족소유,70000000.0,대환대출,NaN,NaN,7.0,71000000.0
899972,417912,141,NaN,NaN,2022-06-15 21:03:58,600.0,24000000.0,EARNEDINCOME,20120701.0,계약직,전월세,50000000.0,생활비,0.0,NaN,5.0,51000000.0
896885,550351,141,NaN,NaN,2022-06-15 22:05:19,600.0,24000000.0,EARNEDINCOME,20120101.0,계약직,전월세,500000000.0,대환대출,0.0,NaN,5.0,51000000.0
82441,22899,141,NaN,NaN,2022-06-15 22:16:05,600.0,25000000.0,EARNEDINCOME,20120701.0,계약직,전월세,40000000.0,생활비,0.0,NaN,5.0,51000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080598,1735632,879368,NaN,NaN,2022-05-09 09:34:32,200.0,30000000.0,OTHERINCOME,NaN,기타,전월세,3000000.0,생활비,0.0,NaN,1.0,13000000.0
1019979,150022,879368,NaN,NaN,2022-05-11 07:38:57,200.0,30000000.0,OTHERINCOME,NaN,기타,전월세,5000000.0,생활비,0.0,NaN,1.0,13000000.0
1069703,656400,879368,NaN,NaN,2022-05-14 16:28:06,200.0,45000000.0,OTHERINCOME,NaN,기타,전월세,100000000.0,생활비,1.0,0.0,1.0,13000000.0
1027934,1136627,879591,NaN,NaN,2022-03-23 20:20:05,650.0,23000000.0,EARNEDINCOME,202107.0,정규직,전월세,30000000.0,생활비,NaN,NaN,NaN,NaN


gender 처리

In [ ]:
spec2['gender'] = spec2['gender'].fillna(spec2.groupby('user_id')['gender'].transform('mean'))

In [ ]:
len(spec1[spec1.gender.isnull()])-len(spec2[spec2.gender.isnull()])

#gender 결측값 3236개 처리

3236

In [ ]:
spec2.loc[spec2.gender.isnull(), 'gender'] = 2

# 결측값을 채울수없는 값은 2로 두어 한범주로 묶는다

birth_year 처리

In [ ]:
spec2['birth_year'] = spec2['birth_year'].fillna(spec2.groupby('user_id')['birth_year'].transform('mean'))

In [ ]:
len(spec1[spec1.birth_year.isnull()])-len(spec2[spec2.birth_year.isnull()])

#birth_year 결측값 3236개 처리

3236

birth_year 결측치 처리를 위해 age로 변환

In [ ]:
spec2['age'] = spec2['birth_year'].apply(lambda x: 2022 - x)

In [ ]:
age_bins = list(range(10,110,10))

In [ ]:
str(age_bins[1]) + '대'

'20대'

In [ ]:
def age_cat(age):
  res = ""
  for i in range(len(age_bins)):
    if age < age_bins[i]:
      res = str(age_bins[i-1]) + '대'
      break
    elif pd.isna(age):
      res = 'missing'
      break
  return(res)

In [ ]:
spec2['age_cate'] = spec2['age'].apply(age_cat)

# age를 10년단위로 범주화, 결측값은 missing으로 처리

In [ ]:
spec2['age_cate']

1290534    20대
1290535    20대
971242     30대
1150725    20대
355849     20대
          ... 
691957     30대
1181092    20대
289906     20대
60175      20대
403673     60대
Name: age_cate, Length: 1394131, dtype: object

##credit_score

결측치 처리

In [ ]:
spec3 = spec2.copy()

In [ ]:
spec3.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                            105112
yearly_income                                5
income_type                                  0
company_enter_month                     171675
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198540
existing_loan_amt                       313752
age                                       9723
age_cate                                     0
dtype: int64

In [ ]:
def change_id(dt, col_name):
  data = dt
  na_id = dt[dt[col_name].isnull()]['user_id']
  normal_id = dt[~dt[col_name].isnull()]['user_id']
  return_id = list(set(na_id) & set(normal_id))
  return_id.sort()
  return(return_id)

#결측치를 대체할 수 있는 user_id 추출 함수

In [ ]:
change_credit_id = change_id(dt = spec3, col_name = 'credit_score')
#credit 결측치를 대체할수 있는 user_id추출

In [ ]:
'''
check_list = []
for i in range(len(change_credit_id)):
  check_list.append(spec3[spec3.user_id == change_credit_id[i]].sort_values(['insert_time']).tail(1)['credit_score'])
'''

"\ncheck_list = []\nfor i in range(len(change_credit_id)):\n  check_list.append(spec3[spec3.user_id == change_credit_id[i]].sort_values(['insert_time']).tail(1)['credit_score'])\n"

In [ ]:
'''
pd.isnull(check_list).sum()
'''

'\npd.isnull(check_list).sum()\n'

In [ ]:
'''
spec3['credit_score'] = spec3.groupby(['user_id'])['credit_score'].apply(lambda x : x.bfill()), 시간이 오래걸린다
'''

"\nspec3['credit_score'] = spec3.groupby(['user_id'])['credit_score'].apply(lambda x : x.bfill()), 시간이 오래걸린다\n"

In [ ]:
temp = spec3[spec3['user_id'].isin(change_credit_id)].groupby('user_id')['credit_score'].apply(lambda x: x.bfill().ffill())
spec3.loc[spec3['user_id'].isin(change_credit_id), 'credit_score'] = temp

#insert_time순으로 결측치 대체

In [ ]:
len(spec2[spec2['credit_score'].isnull()]) - len(spec3[spec3['credit_score'].isnull()])

#credit 결측값 17590 대체

17590

##yearly_income

결측치처리

In [ ]:
spec4 = spec3.copy()

In [ ]:
spec4.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87522
yearly_income                                5
income_type                                  0
company_enter_month                     171675
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198540
existing_loan_amt                       313752
age                                       9723
age_cate                                     0
dtype: int64

In [ ]:
change_income_id = change_id(dt = spec4, col_name = 'yearly_income')

In [ ]:
spec4[spec4['yearly_income'].isnull()]

#user_id 861363의 정보가 부족함(ap_id = 1026915)

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,age,age_cate
597591,391560,329226,1996.0,0.0,2022-06-30 19:27:28,590.0,NaN,OTHERINCOME,NaN,기타,전월세,20000000.0,생활비,0.0,0.0,9.0,27000000.0,26.0,20대
597613,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0,41.0,40대
233316,2045613,702899,1995.0,1.0,2022-06-30 18:37:04,520.0,NaN,FREELANCER,20220630.0,기타,전월세,5000000.0,전월세보증금,0.0,0.0,1.0,1000000.0,27.0,20대
971960,1330905,771592,2003.0,1.0,2022-06-30 18:57:05,750.0,NaN,OTHERINCOME,NaN,기타,자가,3000000.0,생활비,0.0,0.0,1.0,3000000.0,19.0,10대
165422,1026915,861363,1995.0,1.0,2022-06-30 21:11:51,NaN,NaN,OTHERINCOME,NaN,기타,전월세,5000000.0,생활비,0.0,0.0,NaN,NaN,27.0,20대


In [ ]:
print(loan[loan['application_id']==1026915])

#loan에 user_id 861363의 정보가 없기때문에 삭제해도 무방 

spec4 = spec4[spec4['user_id'] != 861363]
#user_id 861363 삭제

Empty DataFrame
Columns: [application_id, loanapply_insert_time, bank_id, product_id, loan_limit, loan_rate, is_applied]
Index: []


In [ ]:
spec4[spec4['user_id'].isin(change_income_id)]

#결측값 대채 가능한지 확인

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,age,age_cate
566127,1547482,329226,1996.0,0.0,2022-06-30 19:21:53,590.0,0.0,OTHERINCOME,NaN,기타,전월세,30000000.0,생활비,0.0,0.0,9.0,27000000.0,26.0,20대
597591,391560,329226,1996.0,0.0,2022-06-30 19:27:28,590.0,NaN,OTHERINCOME,NaN,기타,전월세,20000000.0,생활비,0.0,0.0,9.0,27000000.0,26.0,20대
170413,294001,329226,1996.0,0.0,2022-06-30 19:28:33,590.0,0.0,OTHERINCOME,NaN,기타,전월세,20000000.0,생활비,0.0,0.0,9.0,27000000.0,26.0,20대
597613,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0,41.0,40대
566158,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,기타가족소유,6000000.0,사업자금,0.0,0.0,1.0,3000000.0,41.0,40대
1351402,1287113,702899,1995.0,1.0,2022-05-03 19:33:32,540.0,9000000.0,EARNEDINCOME2,202105.0,정규직,기타가족소유,3000000.0,생활비,0.0,0.0,1.0,1000000.0,27.0,20대
1317821,605126,702899,1995.0,1.0,2022-05-27 18:24:19,470.0,9000000.0,OTHERINCOME,NaN,기타,기타가족소유,1000000.0,생활비,0.0,0.0,1.0,1000000.0,27.0,20대
908289,495894,702899,1995.0,1.0,2022-05-31 22:52:16,470.0,9000000.0,OTHERINCOME,NaN,기타,기타가족소유,1000000.0,생활비,0.0,0.0,1.0,1000000.0,27.0,20대
249635,447815,702899,1995.0,1.0,2022-06-09 14:02:47,540.0,9000000.0,OTHERINCOME,NaN,기타,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,1000000.0,27.0,20대
891745,1901096,702899,1995.0,1.0,2022-06-10 15:06:14,540.0,9000000.0,OTHERINCOME,NaN,기타,전월세,1000000.0,생활비,0.0,0.0,1.0,1000000.0,27.0,20대


In [ ]:
#spec4['yearly_income'] = spec4.groupby('user_id')['yearly_income'].apply(lambda x: x.bfill())

In [ ]:
temp = spec4[spec4.user_id.isin(change_income_id)].groupby('user_id')['yearly_income'].apply(lambda x: x.bfill())
spec4.loc[spec4['user_id'].isin(change_income_id), 'yearly_income'] = temp

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
spec4[spec4['yearly_income'].isnull()]

#yearly_income결측치 처리 완료

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,age,age_cate


In [ ]:
spec4.loc[spec4.yearly_income == 0 & spec4.company_enter_month.isnull(), 'income_type'].value_counts()

# 연수입이 없으면서 입사연월이 없는 자료의 income_type

OTHERINCOME        14996
FREELANCER          1123
PRIVATEBUSINESS      642
EARNEDINCOME         502
EARNEDINCOME2        179
PRACTITIONER          20
Name: income_type, dtype: int64

In [ ]:
spec4.loc[spec4.yearly_income == 0 & spec4.company_enter_month.isnull(), 'income_type'] = 'INOCCUPATION'

# income_type으로 INOCCUPATION(무직) 바꿔준다

이상치 처리

In [ ]:
income_mean = spec4.groupby(['income_type'])['yearly_income'].mean()
print(income_mean)

income_type
EARNEDINCOME       4.370563e+07
EARNEDINCOME2      3.705558e+07
FREELANCER         3.835064e+07
INOCCUPATION       0.000000e+00
OTHERINCOME        3.065390e+07
PRACTITIONER       5.227764e+07
PRIVATEBUSINESS    7.055870e+07
Name: yearly_income, dtype: float64


In [ ]:
income_mean.index

Index(['EARNEDINCOME', 'EARNEDINCOME2', 'FREELANCER', 'INOCCUPATION',
       'OTHERINCOME', 'PRACTITIONER', 'PRIVATEBUSINESS'],
      dtype='object', name='income_type')

In [ ]:
def change_income(income_type):
  res = ""
  for i in range(len(income_mean)):
    if income_type == income_mean.index[i]:
      res = income_mean[i]
      break
  return(res)

In [ ]:
c_i = spec4.loc[spec4.yearly_income >= 1000000000,'income_type'].apply(lambda x: change_income(x))
spec4.loc[spec4.yearly_income >= 1000000000,'yearly_income'] = c_i

# 연봉이 10억이상인 사람은 income_type별 평균 연봉으로 값을 바꿔주었다

In [ ]:
spec4.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87521
yearly_income                                0
income_type                                  0
company_enter_month                     171674
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198539
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
dtype: int64

##company_entrer_month

In [ ]:
spec5 = spec4.copy()

In [ ]:
spec5.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87521
yearly_income                                0
income_type                                  0
company_enter_month                     171674
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198539
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
dtype: int64

In [ ]:
temp_cem = spec5[~spec5['company_enter_month'].isnull()]['company_enter_month'].apply(lambda x: int(str(x)[0:4]))
spec5.loc[~spec5['company_enter_month'].isnull(), 'company_enter_month'] = temp_cem

In [ ]:
change_cem_id = change_id(dt = spec5, col_name = 'company_enter_month')

In [ ]:
spec5[spec5['user_id']==change_cem_id[0]][['user_id','company_enter_month','insert_time']]

,user_id,company_enter_month,insert_time
570808,48,NaN,2022-03-15 21:18:51
296980,48,NaN,2022-04-19 02:56:10
188114,48,NaN,2022-04-26 05:13:21
583327,48,NaN,2022-05-04 04:11:08
302705,48,NaN,2022-05-24 23:25:51
57614,48,2020.0,2022-06-28 04:45:55


In [ ]:
temp = spec5[spec5['user_id'].isin(change_cem_id)].groupby('user_id')['company_enter_month'].apply(lambda x : x.bfill().ffill())
spec5.loc[spec5['user_id'].isin(change_cem_id), 'company_enter_month'] = temp

#insert_time 순으로 na값을 backfill 후 frontfill 시행

In [ ]:
spec5.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87521
yearly_income                                0
income_type                                  0
company_enter_month                     114090
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198539
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
dtype: int64

In [ ]:
spec5.loc[spec5.company_enter_month.isnull(), 'income_type'].value_counts()

# 입사년 정보가 없는데 직장가입자인 경우 확인 필요

OTHERINCOME        102468
INOCCUPATION        11614
EARNEDINCOME            5
PRIVATEBUSINESS         3
Name: income_type, dtype: int64

In [ ]:
spec5.loc[(spec5.company_enter_month.isnull()) & (spec5.income_type=='EARNEDINCOME'), 'income_type'] = 'OTHERINCOME'

이상치 조정

In [ ]:
spec5[spec5.company_enter_month <= spec5.birth_year + 17][['company_enter_month','birth_year','employment_type','yearly_income','income_type']]

# 생일 이전 입사년도가 있는 경우는 이상치로 판단

,company_enter_month,birth_year,employment_type,yearly_income,income_type
55438,2002.0,1985.0,기타,25000000.0,OTHERINCOME
1149205,2002.0,1985.0,기타,25000000.0,FREELANCER
710783,1991.0,1991.0,정규직,73000000.0,EARNEDINCOME
171461,2002.0,1990.0,기타,50000000.0,FREELANCER
602144,2002.0,1990.0,기타,50000000.0,OTHERINCOME
...,...,...,...,...,...
1302787,1970.0,1969.0,정규직,84000000.0,EARNEDINCOME
806238,2006.0,1989.0,기타,0.0,INOCCUPATION
49267,2006.0,1989.0,기타,0.0,INOCCUPATION
394847,2006.0,1989.0,정규직,60000000.0,PRIVATEBUSINESS


In [ ]:
spec5.loc[spec5.company_enter_month <= spec5.birth_year + 17, 'employment_type'] = '기타'
spec5.loc[spec5.company_enter_month <= spec5.birth_year + 17, 'income_type'] = 'OTHERINCOME'
spec5.loc[spec5.company_enter_month <= spec5.birth_year + 17, 'company_enter_month'] = np.nan

# 위에 해당자들 고용형태 => 기타, 근로형태 => OTHERINCOME, 입사연월 => np.nan

company_enter_month 변수 수정  
yyyy-mm => yyyy  
입사연월에서 연차로 변경

In [ ]:
temp_yok = spec5[~spec5['company_enter_month'].isnull()]['company_enter_month'].apply(lambda x: 2023 - x)
spec5.loc[~spec5['company_enter_month'].isnull(), 'year_of_work'] = temp_yok
spec5['year_of_work'].fillna(0, inplace = True)

# 근무년수 변수 생성

In [ ]:
spec5['year_of_work'].fillna(0, inplace = True)

#나머지 정보가 없는 값들은 0으로 변경

In [ ]:
spec5.year_of_work.isnull().sum()

#결측치 115701개 대체

0

## purpose

In [ ]:
spec6 = spec5.copy()

In [ ]:
spec6.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87521
yearly_income                                0
income_type                                  0
company_enter_month                     115701
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198539
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
year_of_work                                 0
dtype: int64

In [ ]:
p_list = spec6['purpose'].unique()
print(p_list)
#같은의미를 갖는 값들이 영어랑 한글로 나뉘어있다
#ex) 생활비 = living, 기타 = 'etc' 등

['생활비' '사업자금' '대환대출' '주택구입' 'SWITCHLOAN' 'LIVING' '투자' '전월세보증금' '기타'
 'BUSINESS' '자동차구입' 'ETC' 'HOUSEDEPOSIT' 'BUYCAR' 'BUYHOUSE' 'INVEST']


In [ ]:
spec6.replace({'purpose':{'기타':'ETC', 
                         '생활비':'LIVING',
                         '자동차구입':'BUYCAR',
                         '투자':'INVEST',
                         '사업자금':'BUSINESS',
                         '주택구입':'BUYHOUSE',
                         '대환대출':'SWITCHLOAN',
                         '전월세보증금':'HOUSEDEPOSIT'}}, inplace = True)

In [ ]:
spec6['purpose'].unique()

array(['LIVING', 'BUSINESS', 'SWITCHLOAN', 'BUYHOUSE', 'INVEST',
       'HOUSEDEPOSIT', 'ETC', 'BUYCAR'], dtype=object)

##personal_rehabilitation_yn
##personal_rehabilitation_complete_yn  


In [ ]:
spec7 = spec6.copy()

In [ ]:
spec7.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87521
yearly_income                                0
income_type                                  0
company_enter_month                     115701
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              587376
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198539
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
year_of_work                                 0
dtype: int64

In [ ]:
spec7['personal_rehabilitation_yn'].unique()

array([ 0., nan,  1.])

In [ ]:
spec7.replace({'personal_rehabilitation_yn' : {0:1, np.nan:0}}, inplace = True)

# 0,1 => 1, nan => 0

In [ ]:
spec7['personal_rehabilitation_yn'].unique()

array([1., 0.])

## existing_loan_cnt

In [ ]:
spec8 = spec7.copy()

In [ ]:
spec8.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                             87521
yearly_income                                0
income_type                                  0
company_enter_month                     115701
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                       198539
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
year_of_work                                 0
dtype: int64

In [ ]:
spec8['existing_loan_cnt'].fillna(0, inplace = True)
spec8[spec8['existing_loan_cnt'].isnull()]

#na값 0으로 대체

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,age,age_cate,year_of_work


## credit na 대체

In [ ]:
spec9 = spec8.copy()

In [ ]:
bins_1 = np.arange(0, 1.1 ,0.1)
print(bins_1)

bins_2 = np.arange(0, 1.2, 0.2)
print(bins_2)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[0.  0.2 0.4 0.6 0.8 1. ]


In [ ]:
q_income = list(spec9['yearly_income'].quantile(bins_1))
q_loan_cnt = list(spec9['existing_loan_cnt'].quantile(bins_2))
print(q_income);print(q_loan_cnt)

[0.0, 18000000.0, 24000000.0, 27000000.0, 30000000.0, 34000000.0, 38000000.0, 44000000.0, 50000000.0, 70000000.0, 990000000.0]
[0.0, 1.0, 2.0, 4.0, 7.0, 278.0]


In [ ]:
cut_income = pd.cut(spec9['yearly_income'], bins = q_income,
                    labels = [1,2,3,4,5,6,7,8,9,10], include_lowest = True)
cut_ecnt = pd.cut(spec9['existing_loan_cnt'], bins = q_loan_cnt,
                  labels = [1,2,3,4,5], include_lowest = True)

spec9['cut_income'] = cut_income
spec9['cut_ecnt'] = cut_ecnt

In [ ]:
spec9[['cut_income', 'cut_ecnt']].isnull().sum()

cut_income    0
cut_ecnt      0
dtype: int64

In [ ]:
m_credit = spec9.groupby(['cut_income','cut_ecnt'])['credit_score'].apply(lambda x : mean(x))
print(m_credit)

cut_income  cut_ecnt
1           1           653.156454
            2           655.690658
            3           633.513316
            4           616.065921
            5           607.593469
2           1           641.824343
            2           645.187183
            3           619.699677
            4           602.916691
            5           598.084800
3           1           640.033312
            2           639.935551
            3           616.363558
            4           599.418448
            5           592.903458
4           1           645.467470
            2           647.566885
            3           625.134916
            4           606.543062
            5           599.674780
5           1           651.220613
            2           651.867229
            3           620.170117
            4           596.731634
            5           593.392957
6           1           649.280462
            2           654.171298
            3           626.750355

In [ ]:
def credit_replace(yearly_income, existing_loan_cnt):
  res = ""
  for i in range(len(q_income)-1):
    if yearly_income <= q_income[i+1]: 
      for j in range(len(q_loan_cnt)-1):
        if existing_loan_cnt <= q_loan_cnt[j+1]:
          res = m_credit.loc[i+1, j+1]
          break
      break    
  return(res)

In [ ]:
temp = spec9[spec9['credit_score'].isnull()]
temp = temp.reset_index(drop = True)

In [ ]:
res = []
for i in range(len(temp['yearly_income'])):
  res.append(credit_replace(temp.loc[i,'yearly_income'], temp.loc[i,'existing_loan_cnt']))

In [ ]:
spec9.loc[spec9['credit_score'].isnull(), 'credit_score'] = res

In [ ]:
spec9.isnull().sum()

application_id                               0
user_id                                      0
birth_year                                9723
gender                                       0
insert_time                                  0
credit_score                                 0
yearly_income                                0
income_type                                  0
company_enter_month                     115701
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn    1203269
existing_loan_cnt                            0
existing_loan_amt                       313751
age                                       9723
age_cate                                     0
year_of_work                                 0
cut_income                                   0
cut_ecnt     

##personal_rehabilitation_yn

In [ ]:
spec10 = spec9.copy()

In [ ]:
spec10.replace({'personal_rehabilitation_yn':{0:'N', 1:'Y'}}, inplace = True)
set(spec10['personal_rehabilitation_yn'])

{'N', 'Y'}

#최종데이터 

사용변수:  
credit_score, yearly_income  
income_type, employment_type  
houseown_type, desired_amount  
purpose, personal_rehabilitation_yn  
existing_loan_cnt,year_of_work, age 

In [ ]:
spec11 = spec10.copy()

In [ ]:
spec11 = spec11[['application_id', 'year_of_work','age_cate','credit_score', 'yearly_income', 'employment_type', 'income_type',
                 'houseown_type', 'desired_amount', 'purpose', 'personal_rehabilitation_yn', 'existing_loan_cnt']]

In [ ]:
spec11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1394130 entries, 1290534 to 403673
Data columns (total 12 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   application_id              1394130 non-null  int64  
 1   year_of_work                1394130 non-null  float64
 2   age_cate                    1394130 non-null  object 
 3   credit_score                1394130 non-null  float64
 4   yearly_income               1394130 non-null  float64
 5   employment_type             1394130 non-null  object 
 6   income_type                 1394130 non-null  object 
 7   houseown_type               1394130 non-null  object 
 8   desired_amount              1394130 non-null  float64
 9   purpose                     1394130 non-null  object 
 10  personal_rehabilitation_yn  1394130 non-null  object 
 11  existing_loan_cnt           1394130 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 13

In [ ]:
p_spec = spec11.copy()

In [ ]:
del spec11
del log

데이터병합  
train, test데이터셋 설정

In [ ]:
lo = loan[~loan.loan_limit.isnull() & ~loan.loan_rate.isnull()]

In [ ]:
pl = pd.merge(p_spec, lo, on=['application_id'])
train = pl[~pl.is_applied.isnull()]

In [ ]:
train.to_csv('train.csv')

In [ ]:
del train

In [ ]:
plr = pd.merge(pl, result, on=['application_id', 'product_id'])
test = plr

In [ ]:
test.to_csv('test.csv')

In [ ]:
del test